In [5]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow as tf
#from tensorflow import keras

In [7]:
#from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout

In [8]:
img_size = (28, 28, 1)
layer_sizes = (128, 64)
n_filters = (32, 64, 128)
kernel_sizes = (7, 3, 3)
pool_size = 2
drop_prob = 0.5

In [10]:
model = Sequential([Convolution2D(filters = 32,
                        kernel_size = (3,3),
                        activation = "relu",
                        input_shape = (28,28,1)),
                    Convolution2D(filters = 32,
                        kernel_size = (3,3),
                        activation = "relu"),
                    MaxPooling2D(pool_size),
                    Dropout(.20),
                    Convolution2D(filters = 64,
                        kernel_size = (3,3),
                        activation = "relu"),
                    Convolution2D(filters = 64,
                        kernel_size = (3,3),
                        activation = "relu"),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(.1),
                    Flatten(),
                    Dense(256, activation="relu"),
                    Dense(units=5, activation="softmax") #output_dim/units is equal to the number of image classes
                   ])

In [16]:
# convolution layer 1
#model.add(Convolution2D(filters = 32,
#                        kernel_size = (3,3),
#                        activation = "relu",
#                        input_shape = (28,28,1)))


In [ ]:
# convolution layer 2
#model.add(Convolution2D(filters = 32,
#                        kernel_size = (3,3),
#                        activation = "relu"))


In [ ]:
# maxpooling layer 1
#model.add(MaxPooling2D(pool_size=(2,2)))


In [ ]:
# deactivating some elements to avoid overfitting
#model.add(Dropout(.20))


In [ ]:
# convolution layer 3
#model.add(Convolution2D(filters = 64,
#                        kernel_size = (3,3),
#                        activation = "relu"))
                        

In [ ]:
# convolution layer 4
#model.add(Convolution2D(filters = 64,
#                        kernel_size = (3,3),
#                        activation = "relu"))


In [ ]:
# maxpooling layer 2
#model.add(MaxPooling2D(pool_size=(2,2)))


In [ ]:
# deactivating some elements
#model.add(Dropout(.1))


In [ ]:
# flattening layer
#model.add(Flatten())

# fully connected layer
#model.add(Dense(256, activation="relu"))
#model.add(Dense(output_dim=5, activation="softmax"))  #output_dim is the number of classes used for classification
                                                      # i.e the image categories  




In [ ]:
model.compile(optimizer = "adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest')



In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'data/train_set',
        target_size=(28, 28),
        color_mode="grayscale",
        batch_size=128,
        class_mode='categorical')



In [ ]:
test_set = test_datagen.flow_from_directory(
        'data/test_set',
        target_size=(28, 28),
        color_mode="grayscale",
        batch_size=128,
        class_mode='categorical') 



In [ ]:
model.fit_generator(
        training_set,
        steps_per_epoch=12000,
        epochs=5,
        validation_data=test_set,
        validation_steps=3000)

model.save("model_new.h5")
model_json = model.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)